In [ ]:
import itertools
import logging
import os.path as osp
import glob
import tempfile

import mmcv
import numpy as np
from mmcv.utils import print_log
from terminaltables import AsciiTable

from mmdet.core import eval_recalls
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.datasets import build_dataset, build_dataloader

In [ ]:
from mmcv import Config

dataset_type = 'TrafficSignDataset'
data_root = '/home/zhanggefan/Downloads/Tencent/det/trainsplit/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(600, 600), keep_ratio=False),
    dict(type='RandomFlip', flip_ratio=0),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(600, 600),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'label/',
        img_prefix=data_root + 'img/',
        pipeline=train_pipeline))
evaluation = dict(interval=1, metric='bbox')

cfg=dict(dataset_type=dataset_type,
         data_root=data_root,
         img_norm_cfg=img_norm_cfg,
         train_pipeline=train_pipeline,
         test_pipeline=test_pipeline,
         data=data,
         evaluation=evaluation)

cfg = Config(cfg)
ds = build_dataset(cfg.data.train)
data_loaders = build_dataloader(ds,1,0,dist=False)

In [ ]:
def draw(fig, img, boxes):
    ax = fig.add_subplot(111)
    ax.invert_yaxis()
    ax.imshow(img)
    for rbbox in boxes:
        x1,y1,x2,y2 = rbbox.tolist()
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, angle=0, linewidth=0.5,
                             facecolor='none', edgecolor='b')
        ax.add_patch(rect)
        ax.plot([(x1+x2)/2], [(y1+y2)/2], '+', c='b', markersize=0.5)


In [ ]:
import matplotlib.pyplot as plt

mean = np.array(img_norm_cfg['mean'])
std = np.array(img_norm_cfg['std'])

for smp in data_loaders:
    img = smp['img'].data[0][0].permute(1,2,0).numpy()
    img = img*std[None,None,:]+mean[None,None,:]
    img = img.round().astype(np.int)
    bbox = smp['gt_bboxes'].data[0][0]
    label = smp['gt_labels'].data[0][0]
    
    fig = plt.figure(dpi=200)
    draw(fig, img, bbox)
    plt.show()